In [21]:
import os
import pandas as pd
import csv
from io import StringIO

In [22]:
def get_event_type(event):
    event = event.strip('()').upper()

    if event == 'GP':
        return 'Grand Prix'
    elif event == 'JO':
        return 'Olympics'
    elif event == 'SA':
        return 'Satellite'
    elif event == 'A':
        return 'World Cup'
    elif event == 'CHZ':
        return 'Zonal Championship'
    elif event == 'CHM':
        return 'World Championship'
    else:
        raise ValueError(f'Unknown event type: {event}')


def get_tournament_data(tournament_data):
    tournament_split = tournament_data.split()
    date = tournament_split[0]
    event_name = ' '.join(tournament_split[1:-1])
    event_type = get_event_type(tournament_split[-1])

    return date, event_name, event_type

### Iterate through each data file

In [23]:
current_dir = os.getcwd()
try:
    results_dir = os.path.join(current_dir, "results")
except FileNotFoundError:
    print("No results directory found.")
    exit(1)

data = []

for results in os.listdir(results_dir):
    results_path = os.path.join(results_dir, results)
    if results_path.endswith('csv'):
        with open(results_path, 'r') as file:
            lines = file.readlines()

            tournaments = lines[3].strip()
            reader = csv.reader(StringIO(tournaments))
            tournaments = list(reader)[0][2:-1]

            for line in lines[4:504]:
                line = line.strip()
                columns = line.split(',')
                current_rank = columns[0]
                fencer_name = columns[1]
                nationality = columns[2]
             
                for i, tournament in enumerate(tournaments):
                    points_index = 3 + i # points start at column 3
                    points = columns[points_index]

                    if points:
                        points = abs(float(points))
                        date, event_name, event_type = get_tournament_data(tournament)
                        data.append([current_rank, fencer_name, nationality, event_name, event_type, points, date])

### Store data to Panda dataframe

In [24]:
df = pd.DataFrame(data, columns=['current_rank', 'fencer_name', 'fencer_nationality', 'event_name', 'event_type', 'points', 'date'])
df = df.drop_duplicates(subset=['fencer_name', 'event_name', 'date'], keep='last')
df['date'] = pd.to_datetime(df['date'])
df.to_csv('data.csv', index=False)
df

/var/folders/v6/48wqz1mn4xj2xrf4w53dnb1c0000gn/T/ipykernel_51593/3922308448.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,current_rank,fencer_name,fencer_nationality,event_name,event_type,points,date
0,1,MASSIALAS Alexander,USA,Bonn,World Cup,2.0,2022-11-11
1,1,MASSIALAS Alexander,USA,Tokyo,World Cup,14.0,2022-09-12
2,1,MASSIALAS Alexander,USA,Paris,World Cup,32.0,2023-12-01
3,1,MASSIALAS Alexander,USA,Turin,Grand Prix,21.0,2023-11-02
4,1,MASSIALAS Alexander,USA,Cairo,World Cup,32.0,2023-02-23
...,...,...,...,...,...,...,...
25785,500,PARCHATKA Niklas,GER,Paris,World Cup,0.0,2013-01-25
25786,500,PARCHATKA Niklas,GER,Bonn,World Cup,0.0,2013-03-22
25787,501,MOLLARD Christopher,GBR,Londres,Satellite,0.0,2012-02-12
25788,502,FERNANDEZ-MIRANDA Jose M.,ESP,La Coruña,World Cup,0.0,2013-02-22
